In [115]:
import folium
from geopy.geocoders import Nominatim
from shapely.geometry import Polygon
from shapely.geometry import Point
import math
import pandas as pd
import re

import geopandas as gpd

In [116]:
def create_list_station():
    df = pd.read_csv('./dataset/regularite-mensuelle-tgv-aqst.csv',sep=";")
    list_station=[]

    for x in df["Gare de départ"]:
        if(x not in list_station):
            list_station.append(x)

    for x in df["Gare d'arrivée"]:
        if(x not in list_station):
            list_station.append(x)

    list_station.sort()

    list_station.remove("GENEVE")
    list_station.remove("FRANCFORT")
    list_station.remove("MADRID")
    list_station.remove("LAUSANNE")
    list_station.remove("STUTTGART")
    list_station.remove("ZURICH")
    list_station.remove("BARCELONA")
    list_station.remove("ITALIE")


    return list_station

list_station = create_list_station()

print(list_station,len(list_station))

['AIX EN PROVENCE TGV', 'ANGERS SAINT LAUD', 'ANGOULEME', 'ANNECY', 'ARRAS', 'AVIGNON TGV', 'BELLEGARDE (AIN)', 'BESANCON FRANCHE COMTE TGV', 'BORDEAUX ST JEAN', 'BREST', 'CHAMBERY CHALLES LES EAUX', 'DIJON VILLE', 'DOUAI', 'DUNKERQUE', 'GRENOBLE', 'LA ROCHELLE VILLE', 'LAVAL', 'LE CREUSOT MONTCEAU MONTCHANIN', 'LE MANS', 'LILLE', 'LYON PART DIEU', 'MACON LOCHE', 'MARNE LA VALLEE', 'MARSEILLE ST CHARLES', 'METZ', 'MONTPELLIER', 'MULHOUSE VILLE', 'NANCY', 'NANTES', 'NICE VILLE', 'NIMES', 'PARIS EST', 'PARIS LYON', 'PARIS MONTPARNASSE', 'PARIS NORD', 'PARIS VAUGIRARD', 'PERPIGNAN', 'POITIERS', 'QUIMPER', 'REIMS', 'RENNES', 'SAINT ETIENNE CHATEAUCREUX', 'ST MALO', 'ST PIERRE DES CORPS', 'STRASBOURG', 'TOULON', 'TOULOUSE MATABIAU', 'TOURCOING', 'TOURS', 'VALENCE ALIXAN TGV', 'VANNES'] 51


In [117]:
def create_dict_tgv():
    df = pd.read_csv('./dataset/regularite-mensuelle-tgv-aqst.csv',sep=";")
    list_station = create_list_station()
    dict_tgv = {}

    for x in df.index:
        gare = df["Gare de départ"][x]
        if (gare in list_station):
            if(gare in dict_tgv):
                dict_tgv[gare][0] += df["Nombre de circulations prévues"][x]
                dict_tgv[gare][1] += df["Nombre de trains en retard au départ"][x]
                dict_tgv[gare][2] += 1
                dict_tgv[gare][3] += df["Retard moyen des trains en retard au départ"][x]
                
            else:
                value = [df["Nombre de circulations prévues"][x] , df["Nombre de trains en retard au départ"][x], 1, df["Retard moyen des trains en retard au départ"][x]]
                dict_tgv[gare] = value

    for x in dict_tgv:
        dict_tgv[x][0] = round((dict_tgv[x][1]/dict_tgv[x][0])*100)
        dict_tgv[x][1] = round(dict_tgv[x][3]/dict_tgv[x][2])
        dict_tgv[x].pop(2)
        dict_tgv[x].pop(2)


    for x in df.index:
        gare = df["Gare d'arrivée"][x]
        if (gare in list_station ):
            if(gare in dict_tgv and len(dict_tgv[gare]) > 2):
                dict_tgv[gare][2] += df["Nombre de circulations prévues"][x]
                dict_tgv[gare][3] += df["Nombre de trains en retard à l'arrivée"][x]
                dict_tgv[gare][4] += 1
                dict_tgv[gare][5] += df["Retard moyen des trains en retard à l'arrivée"][x]
                
            else:
                dict_tgv[gare].append(df["Nombre de circulations prévues"][x])
                dict_tgv[gare].append(df["Nombre de trains en retard à l'arrivée"][x])
                dict_tgv[gare].append(1)
                dict_tgv[gare].append(df["Retard moyen des trains en retard à l'arrivée"][x])


    for x in dict_tgv:
        dict_tgv[x][2] = round((dict_tgv[x][3]/dict_tgv[x][2])*100)
        dict_tgv[x][3] = round(dict_tgv[x][5]/dict_tgv[x][4])
        dict_tgv[x].pop(4)
        dict_tgv[x].pop(4)

    dict_tgv["BELLEGARDE"] = dict_tgv.pop("BELLEGARDE (AIN)")
    dict_tgv["ANGERS ST LAUD"] = dict_tgv.pop("ANGERS SAINT LAUD")
    dict_tgv["BESANCON TGV"] = dict_tgv.pop("BESANCON FRANCHE COMTE TGV")
    dict_tgv["ST ETIENNE CHATEAUCREUX"] = dict_tgv.pop("SAINT ETIENNE CHATEAUCREUX")
    dict_tgv["VALENCE TGV"] = dict_tgv.pop("VALENCE ALIXAN TGV")
    dict_tgv["LE CREUSOT MONTCEAU"] = dict_tgv.pop("LE CREUSOT MONTCEAU MONTCHANIN")
    dict_tgv["PARIS MONTPARNASSE VAUGIRARD"] = dict_tgv.pop("PARIS VAUGIRARD")
    
        
        

    

    return dict_tgv

dict_tgv = create_dict_tgv()

print(dict_tgv,len(dict_tgv))

print(dict_tgv.keys())

{'BORDEAUX ST JEAN': [36, 11, 10, 34], 'LA ROCHELLE VILLE': [18, 8, 8, 34], 'PARIS MONTPARNASSE': [27, 10, 15, 29], 'QUIMPER': [20, 20, 9, 42], 'RENNES': [42, 11, 11, 35], 'ST PIERRE DES CORPS': [47, 6, 12, 20], 'TOURS': [18, 6, 14, 19], 'NANTES': [34, 12, 11, 38], 'PARIS EST': [32, 10, 14, 28], 'STRASBOURG': [51, 7, 8, 39], 'DUNKERQUE': [4, 12, 11, 34], 'LILLE': [36, 8, 17, 33], 'TOURCOING': [15, 14, 15, 40], 'CHAMBERY CHALLES LES EAUX': [64, 12, 15, 35], 'LYON PART DIEU': [58, 9, 13, 35], 'MONTPELLIER': [46, 11, 15, 38], 'MULHOUSE VILLE': [50, 6, 11, 37], 'NICE VILLE': [31, 9, 13, 48], 'PARIS LYON': [19, 14, 14, 37], 'BREST': [19, 16, 8, 44], 'POITIERS': [40, 9, 10, 25], 'TOULOUSE MATABIAU': [22, 15, 13, 47], 'REIMS': [29, 6, 11, 21], 'DOUAI': [38, 14, 16, 22], 'PARIS NORD': [29, 14, 17, 21], 'MACON LOCHE': [74, 12, 9, 31], 'MARNE LA VALLEE': [63, 10, 19, 36], 'PERPIGNAN': [36, 11, 16, 47], 'TOULON': [43, 11, 14, 40], 'METZ': [45, 7, 11, 24], 'NANCY': [36, 7, 7, 30], 'GRENOBLE': [10,

In [118]:
def complete_dict_tgv():
    df = pd.read_csv('./dataset/referentiel-gares-voyageurs.csv',sep=";")
    dict_tgv = create_dict_tgv()
    lis = []

    for gare in dict_tgv:
        for x in df.index:
            if type(df["RG"][x]) == str and ((gare in df["RG"][x]) or gare in df["Intitulé plateforme"][x].upper()) and gare not in lis:
                lis.append(gare)
                dict_tgv[gare].append(df["Latitude"][x])
                dict_tgv[gare].append(df["Longitude"][x])


    return dict_tgv
dict_tgv = complete_dict_tgv()
print(dict_tgv, len(dict_tgv))

{'BORDEAUX ST JEAN': [36, 11, 10, 34, 44.825873, -0.556697], 'LA ROCHELLE VILLE': [18, 8, 8, 34, 46.15269, -1.145305], 'PARIS MONTPARNASSE': [27, 10, 15, 29, 48.841172, 2.320514], 'QUIMPER': [20, 20, 9, 42, 47.994455, -4.092103], 'RENNES': [42, 11, 11, 35, 46.3154922, 3.3941783], 'ST PIERRE DES CORPS': [47, 6, 12, 20, 47.386099, 0.72355], 'TOURS': [18, 6, 14, 19, 47.3541741, 0.6675586], 'NANTES': [34, 12, 11, 38, 47.216148, -1.542356], 'PARIS EST': [32, 10, 14, 28, 48.876742, 2.358424], 'STRASBOURG': [51, 7, 8, 39, 48.5632897, 7.6950455], 'DUNKERQUE': [4, 12, 11, 34, 51.03047, 2.368961], 'LILLE': [36, 8, 17, 33, 50.6119054, 3.075837], 'TOURCOING': [15, 14, 15, 40, 50.71683, 3.16804], 'CHAMBERY CHALLES LES EAUX': [64, 12, 15, 35, 45.571302, 5.919547], 'LYON PART DIEU': [58, 9, 13, 35, 45.760559, 4.859355], 'MONTPELLIER': [46, 11, 15, 38, 43.604738, 3.880674], 'MULHOUSE VILLE': [50, 6, 11, 37, 47.742224, 7.342396], 'NICE VILLE': [31, 9, 13, 48, 43.704556, 7.261904], 'PARIS LYON': [19, 14

In [119]:
def create_map_retard_tgv(dict_tgv):
    
    m = folium.Map(
            location=[45.90, 6.09],
            zoom_start=7,
        )
    for x in dict_tgv:
        text = "<center><b>" + str(x) + "</b><br/><br/>Trains en retard au départ : <b>" + str(dict_tgv[x][0]) + "%</b><br/>Moyenne des retards au départ : <b>" + str(dict_tgv[x][1]) + " min</b><br/>Trains en retard à l'arrivée : <b>" + str(dict_tgv[x][2]) + "%</b><br/>Moyenne des retards à l'arrivée : <b>" + str(dict_tgv[x][3]) + " min</b></center><br/>"
        folium.Circle(
            radius=500,
            location=[dict_tgv[x][4], dict_tgv[x][5]],
            popup = folium.Popup(text, parse_html=False, min_width=300, max_width=300),
            color = "red",
            fill=True,
        ).add_to(m)


    m.save("stations_tgv_retard.html")

create_map_retard_tgv(dict_tgv)

In [120]:
def create_dict_ter():
    df = pd.read_csv('./dataset/regularite-mensuelle-ter.csv',sep=";")
    dict_regions = {}

    for x in df.index:
        region = df["Région"][x]
    
        if ("2022" in df["Date"][x]):
            if(region in dict_regions):

                dict_regions[region][0] += df["Nombre de trains ayant circulé"][x]
                dict_regions[region][1] += df["Nombre de trains annulés"][x]
                dict_regions[region][2] += df["Taux de régularité"][x]
                
                
            else:
                value = [df["Nombre de trains ayant circulé"][x] , df["Nombre de trains annulés"][x],df["Taux de régularité"][x]]
                dict_regions[region] = value

    for x in dict_regions:
        for y in range(len(dict_regions[x])):
            # print(x)
            # print(y)
            dict_regions[x][y] = round(dict_regions[x][y] / 11)

    return dict_regions

dict_ter = create_dict_ter()
print(create_dict_ter(), len(create_dict_ter()))

{'Grand Est': [38841, 927, 95], 'Normandie': [10215, 159, 93], 'Centre Val-de-Loire': [10143, 163, 91], 'Hauts-de-France': [26962, 1322, 89], 'Bretagne': [9008, 100, 96], 'Auvergne-Rhône-Alpes': [36455, 718, 92], 'Bourgogne-Franche-Comté': [14903, 193, 94], 'Occitanie': [12319, 480, 89], 'Nouvelle Aquitaine': [16560, 419, 91], 'Pays-de-la-Loire': [13490, 204, 94], "Provence Alpes Côte d'Azur": [13246, 313, 90]} 11


In [121]:

def create_map_retard_ter(dict_ter):
    
    # Charger les données GeoJSON des régions françaises
    regions = gpd.read_file("./dataset/regions.json")
    # print(regions["nom"])


    # Créer une carte de France centrée sur les coordonnées [46.2276, 2.2137]
    france_map = folium.Map(location=[46.2276, 2.2137], zoom_start=6)
    

    # Ajouter les régions sur la carte
    for i in range(len(regions)):
        
        region = regions.iloc[i]
        if (region["nom"] != "Île-de-France" and region["nom"] != "Corse"):
            text = "<center><b>" +region["nom"] + "</b><br/><br/>Moyenne mensuelle des trains ayant circulé : <b>" + str(dict_ter[region["nom"]][0]) + "</b><br/>Moyenne mensuelle des trains annulés : <b>" + str(dict_ter[region["nom"]][1]) + "</b><br/>Taux de régularité des trains : <b>" + str(dict_ter[region["nom"]][2]) + "%</b></center><br/>"
            geometry = region["geometry"]
            folium.GeoJson(geometry).add_child(folium.Popup(text, parse_html=False, min_width=300, max_width=300)).add_to(france_map)

    # Afficher la carte
    france_map.save("regions_ter_retard.html")

create_map_retard_ter(dict_ter)

In [122]:
def create_map_retard(dict_tgv, dict_ter):
    # style_red = {'fillColor': '#FF0000', 'color': '#FF0000'} #RED
    # style_green = {'fillColor': '#00FF00', 'color': '#00FF00'} #GREEN
    # style_orange = {'fillColor': '#FFA500', 'color': '#FFA500'} #ORANGE
    # # Charger les données GeoJSON des régions françaises
    regions = gpd.read_file("./dataset/regions.json")
    # print(regions["nom"])


    # Créer une carte de France centrée sur les coordonnées [46.2276, 2.2137]
    france_map = folium.Map(location=[46.2276, 2.2137], zoom_start=6)
    

    # Ajouter les régions sur la carte
    for i in range(len(regions)):
        
        region = regions.iloc[i]
        
        if (region["nom"] != "Île-de-France" and region["nom"] != "Corse"):
            # style_color = {'fillColor': '#FFA500', 'color': '#FFA500'}
            text = "<center><b>" +region["nom"] + "</b><br/><br/>Moyenne mensuelle des TER ayant circulé : <b>" + str(dict_ter[region["nom"]][0]) + "</b><br/>Moyenne mensuelle des TER annulés : <b>" + str(dict_ter[region["nom"]][1]) + "</b><br/>Moyenne mensuelle de la proportion des TER arrivant à l'heure : <b>" + str(dict_ter[region["nom"]][2]) + "%</b></center><br/>"
            geometry = region["geometry"]
            # if dict_ter[region["nom"]][2] <= 91:
            #     style_color = {'fillColor': '#FF0000', 'color': '#FF0000'}
            # elif dict_ter[region["nom"]][2] > 94:
            #     style_color = {'fillColor': '#00FF00', 'color': '#00FF00'}
            # else:
            #     style_color = {'fillColor': '#FFA500', 'color': '#FFA500'}

            
            # n=folium.GeoJson(geometry, style_function=lambda x:style_color)
            folium.GeoJson(geometry).add_child(folium.Popup(text, parse_html=False, min_width=300, max_width=300)).add_to(france_map)

    for x in dict_tgv:
        text = "<center><b>" + str(x) + "</b><br/><br/>TGV en retard au départ : <b>" + str(dict_tgv[x][0]) + "%</b><br/>Moyenne des retards TGV au départ : <b>" + str(dict_tgv[x][1]) + " min</b><br/>TGV en retard à l'arrivée : <b>" + str(dict_tgv[x][2]) + "%</b><br/>Moyenne des retards TGV à l'arrivée : <b>" + str(dict_tgv[x][3]) + " min</b></center><br/>"
        folium.Circle(
            radius=500,
            location=[dict_tgv[x][4], dict_tgv[x][5]],
            popup = folium.Popup(text, parse_html=False, min_width=300, max_width=300),
            color = "red",
            fill=True,
        ).add_to(france_map)


    france_map.save("carte_retard.html")
    
create_map_retard(dict_tgv, dict_ter)